In [ ]:
import os
import numpy as np
import random
import glob
from datetime import datetime
import logging
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout, BatchNormalization
from tensorflow.keras import callbacks, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.mixed_precision import set_global_policy
from tensorflow.keras.backend import clear_session
import gc
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from numba import cuda

In [ ]:
# Limpa a sessão antes de construir o modelo
clear_session()

# Configurações de GPU
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

# Define a política de precisão mista
set_global_policy('mixed_float16')

# Libera memória do CUDA
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

In [ ]:
output_path = "modelos" #caminha onde é salvo o modelo
files_per_batch = 1000
labeled_data_dir = "train_graphs" #caminho dos dados para treinar o modelo
param_v_a_1 = [4, 3, 2] #camadas compartilhadas rede bound
param_v_a_2 = [3, 2, 2] #camadas ocultas
param_p_a_1 = [6, 4, 3] #camadas compartilhadas rede brach
param_p_a_2 = [9, 6, 2] #camadas ocultas
param_p_b = 128 #batch size
param_v_b =  128
param_p_l = 1e-4 #taxa de aprendizado
param_v_l = 1e-4
tam = 30
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
# Pegando todos os arquivos do diretório
files = sorted([os.path.basename(ii) for ii in glob.glob(f"{labeled_data_dir}/*.dimacs")])
files_treino, _ = train_test_split(files, test_size=0.1, random_state=42)
files_t, files_v = train_test_split(files_treino, test_size=0.2, random_state=42)

In [ ]:
def parse_file_pointer(fp):
    lines = [ll.strip() for ll in fp]
    ii = 0
    labels = []
    res = []
    cli = []
    numLinhas = 0
    while ii < len(lines):
        line = lines[ii]
        #contando o numero de vertices do grafo
        if "cliqueatual" not in line:
            ii += 1
            numLinhas += 1
            continue

        #pegando a clique atual
        if ii+1 >= len(lines):
            break
        line = line[3:]
        spritado = line.split()
        clique = [int(elem) for elem in spritado[1:]]
        if(numLinhas < tam):
            dif = tam - numLinhas
            clique.extend([0]*dif)
        cli.append(clique)

        #criando o vetor de movimento
        line = lines[ii+1]
        sp = line.split()
        mv = int(sp[-1])
        label = [0] * tam
        label[mv-1] = 1
        labels.append(label)

        #lendo o grafo
        cells = []
        for tt in range(numLinhas, 0, -1):
            cell_line = lines[ii - tt][3:]
            cells.extend([int(float(cc)) for cc in cell_line.split(", ")])
            if(numLinhas < tam):
                dif = tam - numLinhas
                cells.extend([0]*dif)
        while len(cells) < tam * tam:
            cells.extend([0]*tam)
        res.append(cells)
        ii += (numLinhas+2)
    labels_v = list(range(len(labels),0, -1))
    return (res, cli, labels, labels_v)

In [ ]:
def estruturar_entrada(batch_input, batch_labels):
    # Dividir a entrada em uma lista de 151 tensores de forma (batch_size, tam)
    batch_input_list = [batch_input[:, i, :] for i in range(batch_input.shape[1])]
            
    # Converter para tensores do TensorFlow
    x_batch = [tf.convert_to_tensor(tensor, dtype=tf.float32) for tensor in batch_input_list]
    input_dict = {f'input_{i}': tensor for i, tensor in enumerate(x_batch)}
    y_batch = np.array(batch_labels)
    return input_dict, y_batch

In [ ]:
def combinar_entrada(res, clique, labels, remaining_batch_input=[], remaining_batch_labels=[]):
    combined_input = np.array([np.hsplit(np.concatenate([res[i], clique[i]]), tam + 1) for i in range(len(clique))])
    if len(remaining_batch_input) != 0:
        combined_input = np.concatenate((remaining_batch_input, combined_input), axis=0)
        labels = remaining_batch_labels + labels
    return combined_input, labels

In [ ]:
def parse_dir(files):
    res = []
    cli = []
    labels = []
    labels_v = []
    random.seed(42)
    random.shuffle(files)
    random.seed()
    for ff in files:
        with open(os.path.join(labeled_data_dir,ff), 'r') as fp:
            rr, cc, ll, ll_v = parse_file_pointer(fp)
            res.extend(rr)
            cli.extend(cc)
            labels.extend(ll)
            labels_v.extend(ll_v)
    return res, cli, labels, labels_v

In [ ]:
def ler_arquivos(batch_files, value):
    res, clique, labels, labels_v = parse_dir(batch_files)
    if value:
        labels = labels_v
    return res, clique, labels

In [ ]:
def data_generator(batch_size, value, is_train):
    # Gerador de treinamento
    def create_generator(file_list):
        def generator():
            idx = 0
            remaining_batch_input = remaining_batch_labels = []
            while idx < len(file_list):
                batch_files = file_list[idx: idx + files_per_batch]
                res, clique, labels = ler_arquivos(batch_files, value)
                combined_input, labels = combinar_entrada(res, clique, labels, remaining_batch_input, remaining_batch_labels)
                remaining_batch_input = remaining_batch_labels = []
                num_samples = len(combined_input)
                num_batches = num_samples // batch_size
                for batch_idx in range(num_batches):
                    batch_input = combined_input[batch_idx * batch_size: (batch_idx + 1) * batch_size]
                    batch_labels = labels[batch_idx * batch_size: (batch_idx + 1) * batch_size]
                    x_batch, y_batch = estruturar_entrada(batch_input, batch_labels)
                    yield x_batch, y_batch
                idx += files_per_batch
                remaining_samples = num_samples % batch_size
                if remaining_samples > 0:
                    remaining_batch_input = combined_input[-remaining_samples:]
                    remaining_batch_labels = labels[-remaining_samples:]
        return generator

    return create_generator(files_t)() if is_train else create_generator(files_v)()

In [ ]:
def output_signature_type_branch(batch_size):
        output_signature=(
                {f"input_{i}": tf.TensorSpec(shape=(batch_size, tam), dtype=tf.float32) for i in range(tam + 1)},  # Entradas
                tf.TensorSpec(shape=(batch_size, tam), dtype=tf.float32)  # Rótulos
        )
        return output_signature

def output_signature_type_bound(batch_size):
        output_signature=(
                {f"input_{i}": tf.TensorSpec(shape=(batch_size, tam), dtype=tf.float32) for i in range(tam + 1)},  # Entradas
                tf.TensorSpec(shape=(batch_size,), dtype=tf.float32)  # Rótulos
        )
        return output_signature

In [ ]:
class printbatch(callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        logging.info("Epoch: "+ str(epoch))
    def on_epoch_end(self, epoch, logs={}):
        logging.info(logs)

In [ ]:
#treinar rede branch
#verificar se já tem um modelo compilado para treinar
initial_epoch = 0
checkpoint_dir = os.path.join(output_path, "checkpoints", f"dnn_model_{tam}")
os.makedirs(checkpoint_dir, exist_ok=True)
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint_path = os.path.join(checkpoint_dir, "checkpoint_epoch_{epoch:04d}.keras")

if latest_checkpoint:
    print(f"Carregando modelo do checkpoint: {latest_checkpoint}")
    model = tf.keras.models.load_model(latest_checkpoint)
    # Extraindo a última época treinada
    initial_epoch = int(latest_checkpoint.split("_epoch_")[1].split(".keras")[0])

else:
    # Definir camadas de entrada
    inputArray = [Input(shape=(tam,), name=f"input_{i}") for i in range(tam + 1)]

    # Camadas densas compartilhadas
    layer = inputArray
    for i in range(len(param_p_a_1)):
        shared_dense = Dense(tam * param_p_a_1[i], activation='relu')
        layer = [Dropout(0.3)(shared_dense(l)) for l in layer]

    # Concatenar os vetores processados
    merged_vector = Concatenate(axis=-1)(layer)

    #camadas internas
    layer = merged_vector
    for i in range(len(param_p_a_2)):
        layer = Dense(tam*(tam+1)*param_p_a_2[i], activation='relu', kernel_regularizer=l2(1e-3))(layer)
        layer = BatchNormalization()(layer)
        layer = Dropout(0.3)(layer)

    #camada de saida
    output_layer = Dense(tam, activation='softmax')(layer)

    #compilar modelo
    model = Model(inputs=inputArray, outputs=output_layer)
    adam = optimizers.Adam(learning_rate=param_p_l, clipnorm=1.0)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

# Criar os datasets de treino
train_dataset = tf.data.Dataset.from_generator(
    data_generator,  # Gera um novo generator a cada época
    args=(param_p_b, False, True),
    output_signature=output_signature_type_branch(param_p_b)
).prefetch(tf.data.AUTOTUNE)

#dataset de validação
val_dataset = tf.data.Dataset.from_generator(
    data_generator,
    args=(param_p_b, False, False),
    output_signature=output_signature_type_branch(param_p_b)
).prefetch(tf.data.AUTOTUNE)

now = datetime.now()
# Treinar modelo
print("treinamento rede branch iniciado")
model.fit(
    train_dataset,
    epochs=1000,
    initial_epoch=initial_epoch,
    verbose=1,
    validation_data=val_dataset,
    callbacks=[
        printbatch(),
        EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True),
        ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, save_weights_only=False),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-7)
    ]
)
final_model_dnn_path = os.path.join(output_path, f"dnn_model_{tam}.keras")
model.save(final_model_dnn_path)
del model, inputArray, merged_vector, layer, output_layer, train_dataset, val_dataset
print("rede 1 treinada")
clear_session()
cuda.select_device(0)
cuda.close()
cuda.select_device(0)
gc.collect()

In [ ]:
#treinar rede bound
checkpoint_dir = os.path.join(output_path, "checkpoints", f"dnn_value_model_{tam}")
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "checkpoint_epoch_{epoch:04d}.keras")

# Verificar se existe um checkpoint salvo
initial_epoch = 0
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
    print(f"Carregando modelo do checkpoint: {latest_checkpoint}")
    model = tf.keras.models.load_model(latest_checkpoint)
    # Extraindo a última época treinada
    initial_epoch = int(latest_checkpoint.split("_epoch_")[1].split(".keras")[0])

else:
    # Definir camadas de entrada
    inputArray = [Input(shape=(tam,), name=f"input_{i}") for i in range(tam + 1)]

    # Camadas densas compartilhadas
    layer = inputArray
    for i in range(len(param_v_a_1)):
        shared_dense = Dense(tam * param_v_a_1[i], activation='relu')
        layer = [Dropout(0.1)(shared_dense(l)) for l in layer]

    # Concatenar os vetores processados
    merged_vector = Concatenate(axis=-1)(layer)

    #camadas internas
    layer = merged_vector
    for i in range(len(param_v_a_2)):
        layer = Dense(tam*(tam+1)*param_v_a_2[i],activation='relu', kernel_regularizer=l2(1e-4))(layer)
        layer = BatchNormalization()(layer)
        layer = Dropout(0.1)(layer)

    #camada de saida
    output_layer = Dense(1)(layer)

    #compilar modelo
    model = Model(inputs=inputArray, outputs=output_layer)
    adam = optimizers.Adam(learning_rate=param_v_l, clipnorm=1.0)
    model.compile(optimizer=adam, loss='mse', metrics=['mae'])

# Criar os datasets de treino
train_dataset = tf.data.Dataset.from_generator(
    data_generator,  # Gera um novo generator a cada época
    args=(param_v_b, True, True),
    output_signature=output_signature_type_bound(param_v_b)
).prefetch(tf.data.AUTOTUNE)

#dataset de validação
val_dataset = tf.data.Dataset.from_generator(
    data_generator,
    args=(param_v_b, True, False),
    output_signature=output_signature_type_bound(param_v_b)
).prefetch(tf.data.AUTOTUNE)

now = datetime.now()
# Treinar modelo
print("treinamento rede bound iniciado")
model.fit(
    train_dataset,
    epochs=1000,
    initial_epoch=initial_epoch,
    verbose=1,
    validation_data=val_dataset,
    callbacks=[
        printbatch(),
        EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True),
        ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, save_weights_only=False),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-7)
    ]
)
final_model_value_path = os.path.join(output_path, f"dnn_value_model_{tam}.keras")
model.save(final_model_value_path)
del model, inputArray, merged_vector, layer, output_layer, train_dataset, val_dataset
print("rede 2 treinada")
clear_session()
cuda.select_device(0)
cuda.close()
cuda.select_device(0)
gc.collect()